# Exercise - CTD and Implied Repo

#### Notation Commands

$$\newcommand{\Black}{\mathcal{B}}
\newcommand{\Blackcall}{\Black_{\mathrm{call}}}
\newcommand{\Blackput}{\Black_{\mathrm{put}}}
\newcommand{\EcondS}{\hat{S}_{\mathrm{conditional}}}
\newcommand{\Efwd}{\mathbb{E}^{T}}
\newcommand{\Ern}{\mathbb{E}^{\mathbb{Q}}}
\newcommand{\Tfwd}{T_{\mathrm{fwd}}}
\newcommand{\Tunder}{T_{\mathrm{bond}}}
\newcommand{\accint}{A}
\newcommand{\carry}{\widetilde{\cpn}}
\newcommand{\cashflow}{C}
\newcommand{\convert}{\phi}
\newcommand{\cpn}{c}
\newcommand{\ctd}{\mathrm{CTD}}
\newcommand{\disc}{Z}
\newcommand{\done}{d_{1}}
\newcommand{\dt}{\Delta t}
\newcommand{\dtwo}{d_{2}}
\newcommand{\flatvol}{\sigma_{\mathrm{flat}}}
\newcommand{\flatvolT}{\sigma_{\mathrm{flat},T}}
\newcommand{\float}{\mathrm{flt}}
\newcommand{\freq}{m}
\newcommand{\futprice}{\mathcal{F}(t,T)}
\newcommand{\futpriceDT}{\mathcal{F}(t+h,T)}
\newcommand{\futpriceT}{\mathcal{F}(T,T)}
\newcommand{\futrate}{\mathscr{f}}
\newcommand{\fwdprice}{F(t,T)}
\newcommand{\fwdpriceDT}{F(t+h,T)}
\newcommand{\fwdpriceT}{F(T,T)}
\newcommand{\fwdrate}{f}
\newcommand{\fwdvol}{\sigma_{\mathrm{fwd}}}
\newcommand{\fwdvolTi}{\sigma_{\mathrm{fwd},T_i}}
\newcommand{\grossbasis}{B}
\newcommand{\hedge}{\Delta}
\newcommand{\ivol}{\sigma_{\mathrm{imp}}}
\newcommand{\logprice}{p}
\newcommand{\logyield}{y}
\newcommand{\mat}{(n)}
\newcommand{\nargcond}{d_{1}}
\newcommand{\nargexer}{d_{2}}
\newcommand{\netbasis}{\tilde{\grossbasis}}
\newcommand{\normcdf}{\mathcal{N}}
\newcommand{\notional}{K}
\newcommand{\pfwd}{P_{\mathrm{fwd}}}
\newcommand{\pnl}{\Pi}
\newcommand{\price}{P}
\newcommand{\probexer}{\hat{\mathcal{P}}_{\mathrm{exercise}}}
\newcommand{\pvstrike}{K^*}
\newcommand{\refrate}{r^{\mathrm{ref}}}
\newcommand{\rrepo}{r^{\mathrm{repo}}}
\newcommand{\spotrate}{r}
\newcommand{\spread}{s}
\newcommand{\strike}{K}
\newcommand{\swap}{\mathrm{sw}}
\newcommand{\swaprate}{\cpn_{\swap}}
\newcommand{\tbond}{\mathrm{fix}}
\newcommand{\ttm}{\tau}
\newcommand{\value}{V}
\newcommand{\vega}{\nu}
\newcommand{\years}{\tau}
\newcommand{\yearsACT}{\tau_{\mathrm{act/360}}}
\newcommand{\yield}{Y}$$

---

# 1. Cheapest-to-Deliver Analysis

### Data

The file `data/fut_bond_data_FVM5_2025-02-25.xlsx` contains market data on:
- `future` sheet: Information on the 5-year Treasury note future (FVM5)
- `bonds` sheet: Details of all deliverable Treasury notes

The quote date is `2025-02-25`.

In [1]:
import pandas as pd
import numpy as np

In [6]:
futtick = 'FVM5'
DATE = '2025-02-25'
FILE_DATA = f'fut_bond_data_{futtick}_{DATE}.xlsx'

# Load the data
futdata = pd.read_excel(FILE_DATA, sheet_name='future').set_index('field').iloc[:, 0]
bonds = pd.read_excel(FILE_DATA, sheet_name='bonds').set_index('ticker')

In [7]:
# Display futures info
display(futdata)

field
last_update_dt         2025-02-26 00:00:00
px_last                          107.40625
last_tradeable_dt      2025-06-30 00:00:00
fut_dlv_dt_last        2025-07-03 00:00:00
fut_days_expire                        124
fut_ctd                   T 3.625 08/31/29
fut_ctd_px                       97.929688
fut_ctd_gross_basis              -5.949538
fut_ctd_net_basis                 1.543928
Name: FVM5 Comdty, dtype: object

In [8]:
# Display deliverable bonds
display(bonds)

,last_update_dt,px_last,maturity,days_to_mty,dur_adj_mid,cpn,nxt_cpn_dt,days_to_next_coupon,int_acc,accrued_days_between_cpn_dates,days_acc,basis_mid,repo_implied_reporate,repo_reporate,conversion
ticker,,,,,,,,,,,,,,,
91282CLK Govt,2025-02-26,97.929688,2029-08-31,1646,4.031767,3.625,2025-02-28,1,1.802486,181,180,10.834997,4.069133,4.37,0.9135
91282CLN Govt,2025-02-26,97.343750,2029-09-30,1676,4.125978,3.500,2025-03-31,32,1.442308,182,150,13.850500,4.053931,4.37,0.9074
91282CLR Govt,2025-02-26,99.937500,2029-10-31,1707,4.153999,4.125,2025-04-30,62,1.356008,181,119,17.377000,4.074078,4.37,0.9293
91282CMA Govt,2025-02-26,99.960938,2029-11-30,1737,4.236733,4.125,2025-05-31,93,1.008585,182,89,22.374000,4.046093,4.37,0.9281
91282CMD Govt,2025-02-26,101.007812,2029-12-31,1768,4.298372,4.375,2025-06-30,123,0.700967,181,58,24.959000,4.077208,4.37,0.9367
91282CMG Govt,2025-02-26,100.468750,2030-01-31,1799,4.392523,4.250,2025-07-31,154,0.316989,181,27,29.131000,4.078429,4.37,0.9307


### 1.1 Delivery Cost

For each deliverable bond, calculate the **delivery cost** to a short position:

$$\text{Delivery Cost}^i = P^i_t - \convert^i \cdot \futprice$$

where:
- $P^i_t$ is the bond's clean price
- $\convert^i$ is the conversion factor
- $\futprice$ is the futures price

Which bond has the lowest delivery cost?

In [14]:
from datetime import date
date_now = date(2025, 2, 25)


def delivery_cost(P_t, convert, futprice):
    return P_t - convert * futprice

bonds['delivery_cost'] = delivery_cost(bonds['px_last'], bonds['conversion'], futdata['px_last'])
# bonds.sort_values(by='delivery_cost', ascending=True)

print("lowest delivery cost:")
bonds.sort_values(by='delivery_cost', ascending=True).iloc[0]


lowest delivery cost:


last_update_dt                    2025-02-26 00:00:00
px_last                                     97.929688
maturity                          2029-08-31 00:00:00
days_to_mty                                      1646
dur_adj_mid                                  4.031767
cpn                                             3.625
nxt_cpn_dt                        2025-02-28 00:00:00
days_to_next_coupon                                 1
int_acc                                      1.802486
accrued_days_between_cpn_dates                    181
days_acc                                          180
basis_mid                                   10.834997
repo_implied_reporate                        4.069133
repo_reporate                                    4.37
conversion                                     0.9135
delivery_cost                               -0.185922
Name: 91282CLK Govt, dtype: object

### 1.2 Identify the CTD

Calculate the **delivery price ratio** for each bond:

$$\text{Delivery Price}^i = \frac{P^i_t}{\convert^i}$$

The CTD is the bond with the **lowest** delivery price ratio.

Does this match the CTD identified by Bloomberg in the `fut_ctd` field?

In [25]:
def delivery_price(P_t, convert):
    return P_t / convert

bonds['delivery_price'] = delivery_price(bonds['px_last'], bonds['conversion'])
bonds.sort_values(by='delivery_price', ascending=True)

print("lowest delivery price:", bonds.sort_values(by='delivery_price', ascending=True).iloc[0]['delivery_price'])
print("maturity:", bonds.sort_values(by='delivery_price', ascending=True).iloc[0]['maturity'])
print("coupon:", bonds.sort_values(by='delivery_price', ascending=True).iloc[0]['cpn'])
print("")
print("CTD:", futdata['fut_ctd'])
print("")
print("It does!")

lowest delivery price: 107.20272304324028
maturity: 2029-08-31 00:00:00
coupon: 3.625

CTD: T 3.625 08/31/29

It does!


In [26]:
bonds.sort_values(by='delivery_price', ascending=True)

,last_update_dt,px_last,maturity,days_to_mty,dur_adj_mid,cpn,nxt_cpn_dt,days_to_next_coupon,int_acc,accrued_days_between_cpn_dates,days_acc,basis_mid,repo_implied_reporate,repo_reporate,conversion,delivery_cost,delivery_price
ticker,,,,,,,,,,,,,,,,,
91282CLK Govt,2025-02-26,97.929688,2029-08-31,1646,4.031767,3.625,2025-02-28,1,1.802486,181,180,10.834997,4.069133,4.37,0.9135,-0.185922,107.202723
91282CLN Govt,2025-02-26,97.343750,2029-09-30,1676,4.125978,3.500,2025-03-31,32,1.442308,182,150,13.850500,4.053931,4.37,0.9074,-0.116681,107.277661
91282CLR Govt,2025-02-26,99.937500,2029-10-31,1707,4.153999,4.125,2025-04-30,62,1.356008,181,119,17.377000,4.074078,4.37,0.9293,0.124872,107.540622
91282CMA Govt,2025-02-26,99.960938,2029-11-30,1737,4.236733,4.125,2025-05-31,93,1.008585,182,89,22.374000,4.046093,4.37,0.9281,0.277197,107.704921
91282CMD Govt,2025-02-26,101.007812,2029-12-31,1768,4.298372,4.375,2025-06-30,123,0.700967,181,58,24.959000,4.077208,4.37,0.9367,0.400378,107.833685
91282CMG Govt,2025-02-26,100.468750,2030-01-31,1799,4.392523,4.250,2025-07-31,154,0.316989,181,27,29.131000,4.078429,4.37,0.9307,0.505753,107.949662


### 1.3 CTD Intuition

Looking at the deliverable bonds:
- What are the coupons and maturities of the bonds closest to being CTD?
- In the current rate environment (rates around 4-5%), do you expect high-coupon or low-coupon bonds to be CTD?
- Why does CBOT use 6% as the notional rate for conversion factors?

# - - -
- Maturies that are closests to the edge of the basket with coupond around the markets yield.
- low-coupon bonds
- 6% is the standardizing yield
# - - -

---

# 2. Implied Repo Rate

### 2.1 Calculate Implied Repo

For each deliverable bond, calculate the **implied repo rate**:

#### Tip
You can compute accrued interest at delivery using the days to delivery and coupon information in the data.

In [47]:
from datetime import date

from datetime import date

date_now = date(2025, 2, 25)
delivery_date = pd.to_datetime(futdata["fut_dlv_dt_last"]).date()

days_to_delivery = (delivery_date - date_now).days
yearsACT = days_to_delivery / 360

F = float(futdata["px_last"])      # futures price
P_t = bonds["px_last"]              # clean price
cf = bonds["conversion"]            # conversion factor

face = 100.0
coupon_per_period = bonds["cpn"] / 100 * face / 2   # Semi-annual coupon payment

# --- 1) Accrued interest at t (today) ---
# Use the provided accrued interest from Bloomberg
bonds["acc_int_t"] = bonds["int_acc"]




# --- 2) Accrued interest at T (delivery) ---
days_acc = bonds["days_acc"]  # Days since last coupon at time t
days_in_period = bonds["accrued_days_between_cpn_dates"]  # Length of current coupon period
days_to_next_coupon = bonds["days_to_next_coupon"]  # Days until next coupon payment

# Check if next coupon occurs before delivery
coupon_before_delivery = days_to_next_coupon <= days_to_delivery

# Case 1: No coupon payment before delivery
# Accrued interest continues to grow in the same coupon period
mask_no_coupon = ~coupon_before_delivery
bonds.loc[mask_no_coupon, "acc_int_T"] = coupon_per_period[mask_no_coupon] * (
    (days_acc[mask_no_coupon] + days_to_delivery) / days_in_period[mask_no_coupon]
)

# Case 2: Coupon payment occurs before delivery
# After coupon payment, accrued interest resets to 0, then accrues from coupon date to delivery
mask_with_coupon = coupon_before_delivery
days_from_coupon_to_delivery = days_to_delivery - days_to_next_coupon[mask_with_coupon]

# For the next coupon period after the payment, use the same period length
# (semi-annual bonds typically have similar period lengths)
bonds.loc[mask_with_coupon, "acc_int_T"] = coupon_per_period[mask_with_coupon] * (
    days_from_coupon_to_delivery / days_in_period[mask_with_coupon]
)

# --- 3) Dirty price today and invoice at delivery ---
bonds["dirty_price_t"] = P_t + bonds["acc_int_t"]
bonds["invoice_T"] = F * cf + bonds["acc_int_T"]

# --- 4) Implied repo from cell 2.1 formula ---
# Calculate as decimal first, then convert to percentage (to match repo_reporate format)
bonds["implied_repo"] = (1 / yearsACT) * (
    bonds["invoice_T"] / bonds["dirty_price_t"] - 1
) * 100  # Convert to percentage

# Display results
cols = ["cpn", "days_acc", "acc_int_t", "acc_int_T", 
        "px_last", "dirty_price_t", "invoice_T", "implied_repo"]
display(bonds[cols].head())

# Find highest implied repo
highest_repo_bond = bonds["implied_repo"].idxmax()
print(f"\nHighest implied repo: {highest_repo_bond} ({bonds.loc[highest_repo_bond, 'implied_repo']:.4f}%)")

,cpn,days_acc,acc_int_t,acc_int_T,px_last,dirty_price_t,invoice_T,implied_repo
ticker,,,,,,,,
91282CLK Govt,3.625,180,1.802486,1.271754,97.929688,99.732174,99.387364,-0.972383
91282CLN Govt,3.500,150,1.442308,0.923077,97.343750,98.786058,98.383508,-1.146083
91282CLR Govt,4.125,119,1.356008,0.752072,99.937500,101.293508,100.564700,-2.023598
91282CMA Govt,4.125,89,1.008585,0.396635,99.960938,100.969523,100.080375,-2.476715
91282CMD Govt,4.375,58,0.700967,0.060428,101.007812,101.708779,100.667863,-2.878393



Highest implied repo: 91282CMG Govt (2.7822%)


### 2.2 Compare to Actual Repo

The data includes the actual repo rate for each bond (in the `repo_reporate` column).

For each bond, calculate the **repo spread**:

$$\text{Repo Spread}^i = r^i_{\text{IR}} - r^i_{\text{repo}}$$

- Which bond has the highest implied repo?
- Is the implied repo higher or lower than actual repo for the CTD bond?
- What does this tell you about futures pricing?

In [51]:
# Calculate repo spread: Repo Spread = Implied Repo - Actual Repo
bonds['repo_spread'] = bonds['implied_repo'] - bonds['repo_reporate']

# Display results
cols = ['implied_repo', 'repo_reporate', 'repo_spread']
display(bonds[cols].sort_values('implied_repo', ascending=False))

# Find CTD bond (lowest delivery price ratio)
ctd_bond = bonds['delivery_price'].idxmin()

,implied_repo,repo_reporate,repo_spread
ticker,,,
91282CMG Govt,2.782228,4.37,-1.587772
91282CLK Govt,-0.972383,4.37,-5.342383
91282CLN Govt,-1.146083,4.37,-5.516083
91282CLR Govt,-2.023598,4.37,-6.393598
91282CMA Govt,-2.476715,4.37,-6.846715
91282CMD Govt,-2.878393,4.37,-7.248393


# - - - 
- 91282CMG Govt has the highest implied repo at 2.78%
- lower than actual repo
- the futures price is elevated by the short’s delivery options, which is why implied repo is negative for the CTD and lower than actual repo
# - - - 

### 2.3 Interpret Implied Repo

The implied repo can be interpreted as the **annualized return** from:
1. Buying the bond at the dirty price
2. Shorting the futures contract
3. Delivering the bond at expiration

If implied repo > actual repo, what trading opportunity exists?

Why might a bond have a very high implied repo but NOT be the CTD?

# - - -
- Arbitrage opportunity: Buy the bond, short the futures, finance at the actual repo rate, and deliver at expiration
- Conversion factor mismatch: High implied repo means the futures price is low relative to that bond’s spot price. CTD is determined by the delivery price ratio (clean price / conversion factor). A bond can have high implied repo if its conversion factor is unfavorable
# - - -

---

In [ ]:
from datetime import date
import numpy as np

# Conversion factors are calculated from the first day of the delivery month
# For simplicity, we'll use the delivery date as reference
delivery_date = pd.to_datetime(futdata["fut_dlv_dt_last"]).date()

# Get maturity dates and coupon rates
maturity_dates = pd.to_datetime(bonds['maturity']).dt.date
coupon_rates = bonds['cpn'] / 100  # Convert to decimal (e.g., 3.625% -> 0.03625)

# Calculate number of semiannual periods from delivery date to maturity
# Round to nearest semiannual period
days_to_maturity = [(maturity - delivery_date).days for maturity in maturity_dates]
n_periods = np.round(np.array(days_to_maturity) / 182.5).astype(int)  # Approximate semiannual periods

# Calculate conversion factor using the formula:
# CF = (c/0.06) * (1 - 1/(1.03)^n) + 1/(1.03)^n
# where c is coupon rate (decimal) and n is number of semiannual periods
bonds['conversion_calculated'] = (coupon_rates / 0.06) * (1 - 1 / (1.03 ** n_periods)) + (1 / (1.03 ** n_periods))

# Compare calculated vs Bloomberg conversion factors
comparison = pd.DataFrame({
    'ticker': bonds.index,
    'coupon': bonds['cpn'],
    'maturity': bonds['maturity'],
    'conversion_bloomberg': bonds['conversion'],
    'conversion_calculated': bonds['conversion_calculated'],
    'difference': bonds['conversion_calculated'] - bonds['conversion']
})

display(comparison[['ticker', 'coupon', 'conversion_bloomberg', 'conversion_calculated', 'difference']].round(6))

print(f"\nAverage absolute difference: {comparison['difference'].abs().mean():.6f}")
print(f"Max absolute difference: {comparison['difference'].abs().max():.6f}")

# 3. Conversion Factor Calculation

### 3.1 Calculate Conversion Factors

CBOT calculates conversion factors as the price of the bond assuming a flat 6% yield curve.

For a bond with coupon $\cpn$ and maturity in $n$ semiannual periods:

$$\convert = \frac{\cpn}{0.06}\left(1 - \frac{1}{(1.03)^n}\right) + \frac{1}{(1.03)^n}$$

Calculate the conversion factor for each deliverable bond. Compare to the values provided by Bloomberg.

#### Note
The actual calculation is slightly more complex due to settlement conventions and rounding. Your values should be close but may not match exactly.

In [52]:
from datetime import date
import numpy as np

# Conversion factors are calculated from the first day of the delivery month
# For simplicity, we'll use the delivery date as reference
delivery_date = pd.to_datetime(futdata["fut_dlv_dt_last"]).date()

# Get maturity dates and coupon rates
maturity_dates = pd.to_datetime(bonds['maturity']).dt.date
coupon_rates = bonds['cpn'] / 100  # Convert to decimal (e.g., 3.625% -> 0.03625)

# Calculate number of semiannual periods from delivery date to maturity
# Round to nearest semiannual period
days_to_maturity = [(maturity - delivery_date).days for maturity in maturity_dates]
n_periods = np.round(np.array(days_to_maturity) / 182.5).astype(int)  # Approximate semiannual periods

# Calculate conversion factor using the formula:
# CF = (c/0.06) * (1 - 1/(1.03)^n) + 1/(1.03)^n
# where c is coupon rate (decimal) and n is number of semiannual periods
bonds['conversion_calculated'] = (coupon_rates / 0.06) * (1 - 1 / (1.03 ** n_periods)) + (1 / (1.03 ** n_periods))

# Compare calculated vs Bloomberg conversion factors
comparison = pd.DataFrame({
    'ticker': bonds.index,
    'coupon': bonds['cpn'],
    'maturity': bonds['maturity'],
    'conversion_bloomberg': bonds['conversion'],
    'conversion_calculated': bonds['conversion_calculated'],
    'difference': bonds['conversion_calculated'] - bonds['conversion']
})

display(comparison[['ticker', 'coupon', 'conversion_bloomberg', 'conversion_calculated', 'difference']].round(6))

print(f"\nAverage absolute difference: {comparison['difference'].abs().mean():.6f}")
print(f"Max absolute difference: {comparison['difference'].abs().max():.6f}")

,ticker,coupon,conversion_bloomberg,conversion_calculated,difference
ticker,,,,,
91282CLK Govt,91282CLK Govt,3.625,0.9135,0.916641,0.003141
91282CLN Govt,91282CLN Govt,3.500,0.9074,0.912254,0.004854
91282CLR Govt,91282CLR Govt,4.125,0.9293,0.927005,-0.002295
91282CMA Govt,91282CMA Govt,4.125,0.9281,0.927005,-0.001095
91282CMD Govt,91282CMD Govt,4.375,0.9367,0.936738,0.000038
91282CMG Govt,91282CMG Govt,4.250,0.9307,0.931872,0.001172



Average absolute difference: 0.002099
Max absolute difference: 0.004854


### 3.2 Conversion Factor Bias

The 6% notional rate creates a systematic bias:

- When rates are **below 6%**: Which bonds (high or low coupon) tend to be CTD?
- When rates are **above 6%**: Which bonds tend to be CTD?

Given current rate levels, explain which bonds you expect to be close to CTD.

# - - - 
**high duration bonds are penalized more as they adjusted based on the 6% rate. If rates are below 6% say at 4% the 2% difference get accumulated over time. Think 2Y vs 30Y. With 30Y the discounting at the 2% difference makes too much of a difference**


- BELOW: Low-coupon bonds tend to be CTD.
- ABOVE: High-coupon bonds

With rates around 4-5% (below 6%), low-coupon bonds are expected to be close to CTD.
# - - - 

---

# 4. Quality Option Value

### 4.1 Net Basis as Option Value

The **net basis** of the CTD bond can be interpreted as the value of the short's delivery options.

Using the net basis values:
- What is the net basis of the CTD bond?
- How does this compare to the net basis of other deliverable bonds?

#### Tip: Net Basis Calculation

$$\netbasis^i = \fwdprice^i - \convert^i \cdot \futprice$$

or equivalently:

$$\netbasis^i = \grossbasis^i - \text{Carry}^i$$

In [58]:
# Calculate Net Basis
# Net Basis = Forward Price - Conversion Factor × Futures Price
# Or: Net Basis = Gross Basis - Carry

from datetime import date

# Get dates and time to delivery (needed for calculations)
date_now = date(2025, 2, 25)
delivery_date = pd.to_datetime(futdata["fut_dlv_dt_last"]).date()
days_to_delivery = (delivery_date - date_now).days
yearsACT = days_to_delivery / 360

F = float(futdata["px_last"])  # Futures price
P_t = bonds["px_last"]  # Clean spot price
cf = bonds["conversion"]  # Conversion factor

# Method 1: Using Forward Price formula
# Forward Price = (Dirty Price Today) × (1 + repo × ACT/360) - Accrued Interest at Delivery
# Use actual repo rate for forward price calculation
repo_rate = bonds["repo_reporate"] / 100  # Convert to decimal
forward_price = (bonds["dirty_price_t"] * (1 + repo_rate * yearsACT)) - bonds["acc_int_T"]
bonds["net_basis"] = forward_price - cf * F

# Method 2: Using Gross Basis - Carry (for verification)
# Gross Basis = Spot Price - Conversion Factor × Futures Price
gross_basis = P_t - cf * F

# Carry = (Coupon Income - Financing Cost) × time
# Carry = (Coupon × Face - Repo × Dirty Price) × ACT/360
face = 100.0
coupon_income = bonds["cpn"] / 100 * face  # Annual coupon
financing_cost = repo_rate * bonds["dirty_price_t"]
carry = (coupon_income - financing_cost) * yearsACT

# Net Basis = Gross Basis - Carry
bonds["net_basis_alt"] = gross_basis - carry

# Store gross_basis and carry in bonds dataframe for display
bonds["gross_basis"] = gross_basis
bonds["carry"] = carry

# Display results
cols = ["cpn", "net_basis", "gross_basis", "carry"]
display(bonds[cols].sort_values("net_basis", ascending=False))

,cpn,net_basis,gross_basis,carry
ticker,,,,
91282CMD Govt,4.375,2.621245,0.400378,-0.024773
91282CMA Govt,4.125,2.457989,0.277197,-0.102175
91282CLR Govt,4.125,2.302684,0.124872,-0.107209
91282CLN Govt,3.500,1.937465,-0.116681,-0.290471
91282CLK Govt,3.625,1.894427,-0.185922,-0.260727
91282CMG Govt,4.250,0.568977,0.505753,-0.054875


### 4.2 Option Interpretation

The short position in Treasury futures has several embedded options:
- **Quality option**: Choose which bond to deliver
- **Timing option**: Choose when to deliver during the delivery month
- **Wildcard option**: Make delivery decision after trading closes

If the net basis of the CTD is positive (which it usually is), what does this imply about the value of these options to the short?

Why doesn't arbitrage drive the net basis to zero?

# - - - 
A positive net basis means the short’s delivery options have value.
The short can:
Choose which bond to deliver (quality option)
Choose when to deliver during the delivery month (timing option)
Make the delivery decision after trading closes (wildcard option)

positive net basis reflects the value of the short’s delivery options, and arbitrage cannot eliminate it because these options are valuable and not perfectly replicable.
# - - - 

***

# - - - 
### Gross basis vs Net basis

Gross basis -> uses spot price

Net basis -> uses forward price (this accounts for the carry). 

and since spot + carry = forward these are related. Also net basis is just gross basis (-) the carry.
# - - - 